# LSASS Memory Credential Dump validation

## Starting Spark session and loading the Dataset

In [1]:
#Starting Spark session

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark.conf.set("spark.sql.caseSensitive", "true")

#Loading Dataset

df = spark.read.json('empire_mimikatz_logonpasswords_2020-08-07103224.json')
df.createTempView('data_table')


In [2]:
df.printSchema()

root
 |-- @timestamp: string (nullable = true)
 |-- @version: string (nullable = true)
 |-- AccessList: string (nullable = true)
 |-- AccessMask: string (nullable = true)
 |-- AccessReason: string (nullable = true)
 |-- AccountName: string (nullable = true)
 |-- AccountType: string (nullable = true)
 |-- ActivityID: string (nullable = true)
 |-- Application: string (nullable = true)
 |-- AuthenticationPackageName: string (nullable = true)
 |-- CallTrace: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Channel: string (nullable = true)
 |-- CommandLine: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- ContextInfo: string (nullable = true)
 |-- CreationUtcTime: string (nullable = true)
 |-- CurrentDirectory: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- DestAddress: string (nullable = true)
 |-- DestPort: string (nullable = true)
 |-- DestinationHostname: string (nullable = true)
 |-- DestinationIp: string (nullable = t

## Query 1

In [5]:
df = spark.sql(
'''
SELECT `@timestamp`, EventTime, Hostname, SubjectUserName, ProcessName, ObjectName, AccessMask, EventID
FROM data_table
WHERE LOWER(Channel) = "security"
    AND (EventID = 4663 OR EventID = 4656)
    AND ObjectName LIKE "%lsass.exe"
    AND NOT SubjectUserName LIKE "%$"
'''
)
df.show(10,False)

+------------------------+-------------------+---------------------------+---------------+---------------------------------------------------------+--------------------------------------------------+----------+-------+
|@timestamp              |EventTime          |Hostname                   |SubjectUserName|ProcessName                                              |ObjectName                                        |AccessMask|EventID|
+------------------------+-------------------+---------------------------+---------------+---------------------------------------------------------+--------------------------------------------------+----------+-------+
|2020-08-07T14:32:57.592Z|2020-08-07 10:32:56|WORKSTATION5.theshire.local|pgustavo       |C:\Windows\System32\WindowsPowerShell\v1.0\powershell.exe|\Device\HarddiskVolume2\Windows\System32\lsass.exe|0x1010    |4656   |
|2020-08-07T14:32:57.592Z|2020-08-07 10:32:56|WORKSTATION5.theshire.local|pgustavo       |C:\Windows\System32\WindowsPowerSh

In [6]:
df.toPandas().head()

,@timestamp,EventTime,Hostname,SubjectUserName,ProcessName,ObjectName,AccessMask,EventID
0,2020-08-07T14:32:57.592Z,2020-08-07 10:32:56,WORKSTATION5.theshire.local,pgustavo,C:\Windows\System32\WindowsPowerShell\v1.0\pow...,\Device\HarddiskVolume2\Windows\System32\lsass...,0x1010,4656
1,2020-08-07T14:32:57.592Z,2020-08-07 10:32:56,WORKSTATION5.theshire.local,pgustavo,C:\Windows\System32\WindowsPowerShell\v1.0\pow...,\Device\HarddiskVolume2\Windows\System32\lsass...,0x10,4663


## Query 2

In [3]:
df = spark.sql(
'''
SELECT `@timestamp`, Hostname, SourceImage, TargetImage, GrantedAccess, SourceProcessGUID
FROM data_table
WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
    AND EventID = 10
    AND TargetImage LIKE "%lsass.exe"
    AND CallTrace LIKE "%UNKNOWN%"
'''
)
df.show(10,False)

+------------------------+---------------------------+---------------------------------------------------------+-----------------------------+-------------+--------------------------------------+
|@timestamp              |Hostname                   |SourceImage                                              |TargetImage                  |GrantedAccess|SourceProcessGUID                     |
+------------------------+---------------------------+---------------------------------------------------------+-----------------------------+-------------+--------------------------------------+
|2020-08-07T14:32:57.589Z|WORKSTATION5.theshire.local|C:\windows\System32\WindowsPowerShell\v1.0\powershell.exe|C:\windows\system32\lsass.exe|0x1010       |{297bc33e-65d0-5f2d-8207-000000000400}|
+------------------------+---------------------------+---------------------------------------------------------+-----------------------------+-------------+--------------------------------------+



In [4]:
df.toPandas().head()

,@timestamp,Hostname,SourceImage,TargetImage,GrantedAccess,SourceProcessGUID
0,2020-08-07T14:32:57.589Z,WORKSTATION5.theshire.local,C:\windows\System32\WindowsPowerShell\v1.0\pow...,C:\windows\system32\lsass.exe,0x1010,{297bc33e-65d0-5f2d-8207-000000000400}


## Query 3

In [5]:
df = spark.sql(
'''
SELECT ProcessGuid,Image, COUNT(DISTINCT ImageLoaded) AS hits
FROM data_table
WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
    AND EventID = 7
    AND ( 
        ImageLoaded LIKE "%samlib.dll"
        OR ImageLoaded LIKE "%vaultcli.dll"
        OR ImageLoaded LIKE "%hid.dll"
        OR ImageLoaded LIKE "%winscard.dll"
        OR ImageLoaded LIKE "%cryptdll.dll"
    )
    AND `@timestamp` BETWEEN "2020-06-00 00:00:00.000" AND "2020-08-20 00:00:00.000"
    GROUP BY ProcessGuid,Image ORDER BY hits DESC LIMIT 10
'''
)
df.show(10,False)

+--------------------------------------+---------------------------------------------------------+----+
|ProcessGuid                           |Image                                                    |hits|
+--------------------------------------+---------------------------------------------------------+----+
|{297bc33e-65d0-5f2d-8207-000000000400}|C:\Windows\System32\WindowsPowerShell\v1.0\powershell.exe|4   |
+--------------------------------------+---------------------------------------------------------+----+



In [6]:
df.toPandas().head()

,ProcessGuid,Image,hits
0,{297bc33e-65d0-5f2d-8207-000000000400},C:\Windows\System32\WindowsPowerShell\v1.0\pow...,4


## Query 4

In [7]:
df = spark.sql(
'''
SELECT a.`@timestamp`, a.Hostname, m.Image, a.SourceProcessGUID
FROM data_table a
INNER JOIN (
    SELECT ProcessGuid,Image, COUNT(DISTINCT ImageLoaded) AS hits
    FROM data_table
    WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
        AND EventID = 7
        AND ( 
            ImageLoaded LIKE "%samlib.dll"
            OR ImageLoaded LIKE "%vaultcli.dll"
            OR ImageLoaded LIKE "%hid.dll"
            OR ImageLoaded LIKE "%winscard.dll"
            OR ImageLoaded LIKE "%cryptdll.dll"
        )
        AND `@timestamp` BETWEEN "2020-06 00:00:00.000" AND "2020-10-20 00:00:00.000"
        GROUP BY ProcessGuid,Image ORDER BY hits DESC LIMIT 10
    ) m
ON a.SourceProcessGUID = m.ProcessGuid
WHERE a.Channel = "Microsoft-Windows-Sysmon/Operational"
    AND a.EventID = 10
    AND a.TargetImage LIKE "%lsass.exe"
    AND a.CallTrace LIKE "%UNKNOWN%"
    AND m.hits >= 3
'''
)
df.show(10,False)

+------------------------+---------------------------+---------------------------------------------------------+--------------------------------------+
|@timestamp              |Hostname                   |Image                                                    |SourceProcessGUID                     |
+------------------------+---------------------------+---------------------------------------------------------+--------------------------------------+
|2020-08-07T14:32:57.589Z|WORKSTATION5.theshire.local|C:\Windows\System32\WindowsPowerShell\v1.0\powershell.exe|{297bc33e-65d0-5f2d-8207-000000000400}|
+------------------------+---------------------------+---------------------------------------------------------+--------------------------------------+



In [8]:
df.toPandas().head()

,@timestamp,Hostname,Image,SourceProcessGUID
0,2020-08-07T14:32:57.589Z,WORKSTATION5.theshire.local,C:\Windows\System32\WindowsPowerShell\v1.0\pow...,{297bc33e-65d0-5f2d-8207-000000000400}
